In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 10 #10:20%, 20: 40%, 30:60%
train_image_s1 = 20 #10:20%, 20: 40%, 30:60%
train_image_s2 = 30 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = ((image_no<train_image)&(ALL_participant_class>2))|((image_no<train_image_s1)&(ALL_participant_class==1))|((image_no<train_image_s2)&(ALL_participant_class==2))
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
sum(Train_participant_class)


array([200., 300., 100., 100., 100.], dtype=float32)

## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

## Speaker 6 Test dataset 

In [15]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [16]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [17]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [18]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [19]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [20]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [21]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [22]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [23]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [24]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
#                                       'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
#                                       'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_training_Data_Size.csv')
# Perfomance
Perfomance                                                        

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,20%,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,20%,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,20%,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,20%,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,20%,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
5,Group,20%,44.229166,0.9705,0.9521,1.000,0.9550,0.9074,0.8796,0.9826,0.9739,0.9633,0.9633,1.00,0.99,0.62,0.456,0.3069
6,Group,20%,44.535541,0.9632,0.9540,0.991,0.9369,0.9259,0.8981,0.9565,0.9652,0.9541,0.9817,0.99,0.99,0.67,0.496,0.3267
7,Group,20%,42.948731,0.9724,0.9595,1.000,0.9550,0.9074,0.8889,0.9913,0.9739,0.9633,0.9908,1.00,0.99,0.67,0.496,0.2574
8,Group,20%,45.444911,0.9724,0.9503,0.991,0.9459,0.9167,0.8796,0.9913,0.9652,0.9633,0.9725,1.00,0.99,0.60,0.552,0.2673
9,Group,20%,42.707019,0.9669,0.9540,0.991,0.9459,0.9167,0.8704,0.9739,0.9652,0.9541,0.9908,1.00,1.00,0.66,0.480,0.4059


# Create model 

In [25]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [26]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs, 
                           {resnet_model.layers[-2].name:Train_participant_class, 
                            resnet_model.layers[-1].name:Train_command_class,
                            resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                            resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {resnet_model.layers[-2].name:Val_participant_class,
                                              resnet_model.layers[-1].name:Val_command_class,
                                              resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                              resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Group",'Size':'mix_20%&s1_60%&s2_40%','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        resnet_model.save('Initial_group_model_mix_20p_s1(40p)_s2(60p)_0608.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('mix_20p_s1(40p)_s2(60p)_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('mix_20p_s1(40p)_s2(60p)_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
13/13 [==============================] - 4s 320ms/step - loss: 3.6081 - participant_output_loss: 1.3137 - command_output_loss: 2.2896 - command_output_1_loss: 4.1801e-04 - participant_output_1_loss: 0.0043 - participant_output_accuracy: 0.4812 - command_output_accuracy: 0.2400 - command_output_1_accuracy: 0.0463 - participant_output_1_accuracy: 0.2713 - val_loss: 3.2813 - val_participant_output_loss: 1.2508 - val_command_output_loss: 2.0303 - val_command_output_1_loss: 1.5159e-04 - val_participant_output_1_loss: 1.3046e-04 - val_participant_output_accuracy: 0.5856 - val_command_output_accuracy: 0.5046 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.4788
Epoch 2/10
13/13 [==============================] - 2s 169ms/step - loss: 2.8636 - participant_output_loss: 0.9232 - command_output_loss: 1.9401 - command_output_1_loss: 9.5907e-05 - participant_output_1_loss: 1.8242e-04 - participant_output_accuracy: 0.6950 - command_output_accuracy: 0.6650

Epoch 3/10
13/13 [==============================] - 2s 176ms/step - loss: 0.7911 - participant_output_loss: 0.1119 - command_output_loss: 0.6790 - command_output_1_loss: 3.7188e-05 - participant_output_1_loss: 4.0124e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9987 - command_output_1_accuracy: 0.0125 - participant_output_1_accuracy: 0.1325 - val_loss: 0.9656 - val_participant_output_loss: 0.2473 - val_command_output_loss: 0.7182 - val_command_output_1_loss: 3.1775e-05 - val_participant_output_1_loss: 5.2002e-05 - val_participant_output_accuracy: 0.9669 - val_command_output_accuracy: 0.9650 - val_command_output_1_accuracy: 0.0092 - val_participant_output_1_accuracy: 0.1492
Epoch 4/10
13/13 [==============================] - 2s 131ms/step - loss: 0.7010 - participant_output_loss: 0.0981 - command_output_loss: 0.6028 - command_output_1_loss: 3.4966e-05 - participant_output_1_loss: 3.7791e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9987

Epoch 5/10
13/13 [==============================] - 3s 199ms/step - loss: 1.9291 - participant_output_loss: 0.4488 - command_output_loss: 1.4801 - command_output_1_loss: 8.6950e-05 - participant_output_1_loss: 6.5992e-05 - participant_output_accuracy: 0.9438 - command_output_accuracy: 0.9438 - command_output_1_accuracy: 0.0025 - participant_output_1_accuracy: 0.2775 - val_loss: 2.0399 - val_participant_output_loss: 0.5852 - val_command_output_loss: 1.4546 - val_command_output_1_loss: 4.1692e-05 - val_participant_output_1_loss: 7.2924e-05 - val_participant_output_accuracy: 0.9079 - val_command_output_accuracy: 0.9208 - val_command_output_1_accuracy: 0.0442 - val_participant_output_1_accuracy: 0.1952
Epoch 6/10
13/13 [==============================] - 3s 195ms/step - loss: 1.7447 - participant_output_loss: 0.3879 - command_output_loss: 1.3567 - command_output_1_loss: 5.2286e-05 - participant_output_1_loss: 5.8115e-05 - participant_output_accuracy: 0.9613 - command_output_accuracy: 0.9588

Epoch 7/10
13/13 [==============================] - 3s 198ms/step - loss: 0.5146 - participant_output_loss: 0.1074 - command_output_loss: 0.4071 - command_output_1_loss: 2.9417e-05 - participant_output_1_loss: 3.0326e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0250 - participant_output_1_accuracy: 0.2250 - val_loss: 0.7174 - val_participant_output_loss: 0.2413 - val_command_output_loss: 0.4761 - val_command_output_1_loss: 2.0981e-05 - val_participant_output_1_loss: 5.3182e-05 - val_participant_output_accuracy: 0.9650 - val_command_output_accuracy: 0.9705 - val_command_output_1_accuracy: 0.0166 - val_participant_output_1_accuracy: 0.2560
Epoch 1/10
13/13 [==============================] - 4s 292ms/step - loss: 3.6000 - participant_output_loss: 1.2889 - command_output_loss: 2.3021 - command_output_1_loss: 0.0038 - participant_output_1_loss: 0.0051 - participant_output_accuracy: 0.4863 - command_output_accuracy: 0.2037 - comma

Epoch 2/10
13/13 [==============================] - 2s 179ms/step - loss: 1.4419 - participant_output_loss: 0.1106 - command_output_loss: 1.3312 - command_output_1_loss: 4.2165e-05 - participant_output_1_loss: 1.7559e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9937 - command_output_1_accuracy: 0.0025 - participant_output_1_accuracy: 0.1437 - val_loss: 1.5795 - val_participant_output_loss: 0.2369 - val_command_output_loss: 1.3426 - val_command_output_1_loss: 3.3527e-05 - val_participant_output_1_loss: 2.1168e-05 - val_participant_output_accuracy: 0.9669 - val_command_output_accuracy: 0.9595 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1418
Epoch 3/10
13/13 [==============================] - 2s 187ms/step - loss: 1.3555 - participant_output_loss: 0.0970 - command_output_loss: 1.2585 - command_output_1_loss: 3.7709e-05 - participant_output_1_loss: 1.7053e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.

...

Epoch 7/10
13/13 [==============================] - 3s 196ms/step - loss: 1.6857 - participant_output_loss: 0.2193 - command_output_loss: 1.4664 - command_output_1_loss: 8.4497e-05 - participant_output_1_loss: 3.0246e-05 - participant_output_accuracy: 0.9887 - command_output_accuracy: 0.9762 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2025 - val_loss: 1.7917 - val_participant_output_loss: 0.3430 - val_command_output_loss: 1.4486 - val_command_output_1_loss: 5.1189e-05 - val_participant_output_1_loss: 3.3126e-05 - val_participant_output_accuracy: 0.9484 - val_command_output_accuracy: 0.9558 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1878
Epoch 8/10
13/13 [==============================] - 2s 150ms/step - loss: 1.5387 - participant_output_loss: 0.1727 - command_output_loss: 1.3659 - command_output_1_loss: 5.4358e-05 - participant_output_1_loss: 2.8470e-05 - participant_output_accuracy: 0.9950 - command_output_accuracy

Epoch 9/10
13/13 [==============================] - 2s 132ms/step - loss: 0.5531 - participant_output_loss: 0.0491 - command_output_loss: 0.5039 - command_output_1_loss: 5.1068e-05 - participant_output_1_loss: 1.9479e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9987 - command_output_1_accuracy: 0.0538 - participant_output_1_accuracy: 0.2025 - val_loss: 0.7337 - val_participant_output_loss: 0.1607 - val_command_output_loss: 0.5730 - val_command_output_1_loss: 3.9219e-05 - val_participant_output_1_loss: 2.7322e-05 - val_participant_output_accuracy: 0.9669 - val_command_output_accuracy: 0.9724 - val_command_output_1_accuracy: 0.0166 - val_participant_output_1_accuracy: 0.1823
Epoch 10/10
13/13 [==============================] - 2s 141ms/step - loss: 0.5036 - participant_output_loss: 0.0458 - command_output_loss: 0.4577 - command_output_1_loss: 3.6684e-05 - participant_output_1_loss: 1.8148e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.998

Epoch 1/10
13/13 [==============================] - 3s 248ms/step - loss: 1.3729 - participant_output_loss: 0.1722 - command_output_loss: 1.2006 - command_output_1_loss: 9.6160e-05 - participant_output_1_loss: 1.8361e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9925 - command_output_1_accuracy: 0.0012 - participant_output_1_accuracy: 0.1725 - val_loss: 1.5190 - val_participant_output_loss: 0.3067 - val_command_output_loss: 1.2123 - val_command_output_1_loss: 4.0370e-05 - val_participant_output_1_loss: 2.1190e-05 - val_participant_output_accuracy: 0.9650 - val_command_output_accuracy: 0.9595 - val_command_output_1_accuracy: 0.0055 - val_participant_output_1_accuracy: 0.2081
Epoch 2/10
13/13 [==============================] - 2s 171ms/step - loss: 1.2665 - participant_output_loss: 0.1520 - command_output_loss: 1.1144 - command_output_1_loss: 8.4027e-05 - participant_output_1_loss: 1.8116e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9962

Epoch 3/10
13/13 [==============================] - 2s 174ms/step - loss: 2.4375 - participant_output_loss: 0.6672 - command_output_loss: 1.7702 - command_output_1_loss: 7.5834e-05 - participant_output_1_loss: 5.9840e-05 - participant_output_accuracy: 0.8575 - command_output_accuracy: 0.7800 - command_output_1_accuracy: 0.1225 - participant_output_1_accuracy: 0.2150 - val_loss: 2.4660 - val_participant_output_loss: 0.7449 - val_command_output_loss: 1.7210 - val_command_output_1_loss: 7.9948e-05 - val_participant_output_1_loss: 4.5495e-05 - val_participant_output_accuracy: 0.8122 - val_command_output_accuracy: 0.8029 - val_command_output_1_accuracy: 0.2063 - val_participant_output_1_accuracy: 0.4604
Epoch 4/10
13/13 [==============================] - 2s 172ms/step - loss: 2.1334 - participant_output_loss: 0.4931 - command_output_loss: 1.6401 - command_output_1_loss: 1.1831e-04 - participant_output_1_loss: 4.1071e-05 - participant_output_accuracy: 0.9162 - command_output_accuracy: 0.8650

Epoch 5/10
13/13 [==============================] - 2s 137ms/step - loss: 0.5694 - participant_output_loss: 0.0467 - command_output_loss: 0.5226 - command_output_1_loss: 3.9036e-05 - participant_output_1_loss: 2.1492e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0213 - participant_output_1_accuracy: 0.1525 - val_loss: 0.7352 - val_participant_output_loss: 0.1514 - val_command_output_loss: 0.5837 - val_command_output_1_loss: 3.0064e-05 - val_participant_output_1_loss: 2.7867e-05 - val_participant_output_accuracy: 0.9669 - val_command_output_accuracy: 0.9724 - val_command_output_1_accuracy: 0.0239 - val_participant_output_1_accuracy: 0.2597
Epoch 6/10
13/13 [==============================] - 2s 167ms/step - loss: 0.5050 - participant_output_loss: 0.0423 - command_output_loss: 0.4627 - command_output_1_loss: 2.7774e-05 - participant_output_1_loss: 2.0682e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

In [27]:
resnet_model = tf.keras.models.load_model('Initial_group_model_mix_20p_s1(40p)_s2(60p)_0608.h5')
predictions = resnet_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_c = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = resnet_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_s = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_c,acc_s)

0.9816 0.9724


In [28]:
Perfomance

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,20%,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,20%,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,20%,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,20%,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,20%,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Group,mix_20%&s1_60%&s2_40%,53.714297,0.9797,0.9687,0.991,0.9550,1.0000,0.9630,0.9826,0.9565,0.9358,0.9817,0.99,0.99,0.69,0.456,0.3366
66,Group,mix_20%&s1_60%&s2_40%,54.407596,0.9761,0.9742,1.000,0.9730,1.0000,0.9722,0.9739,0.9565,0.9174,0.9817,0.99,0.99,0.68,0.440,0.2673
67,Group,mix_20%&s1_60%&s2_40%,49.085995,0.9779,0.9687,0.991,0.9730,1.0000,0.9537,0.9565,0.9565,0.9541,0.9817,0.99,0.98,0.59,0.440,0.2970
68,Group,mix_20%&s1_60%&s2_40%,51.799617,0.9779,0.9724,0.991,0.9820,1.0000,0.9537,0.9478,0.9565,0.9541,0.9817,1.00,0.99,0.70,0.488,0.2673


In [29]:
Perfomance.to_csv('Performance_0608_training_Data_Size.csv',index=False)

In [30]:
best_index

3